In [17]:
import pandas as pd 
import numpy as np

from dash_website import MAIN_CATEGORIES_TO_CATEGORIES, DIMENSIONS

all_correlations = pd.read_feather("../../data/xwas/univariate_results/linear_correlations_Abdomen.feather").set_index("index")
all_correlations.index = pd.MultiIndex.from_tuples(list(map(eval, all_correlations.index.tolist())), names=["category", "variable"])

category = "FamilyHistory"
# category = "All"
main_category = "Biomarkers"
all_correlations

p_value  \
category            variable                                                           
Alcohol             Alcohol intake frequency.                           8.471062e-15   
                    Average weekly red wine intake                      1.261667e-16   
                    Average weekly champagne plus white wine intake     2.510791e-16   
                    Average weekly beer plus cider intake               1.640160e-15   
                    Average weekly spirits intake                       2.763600e-13   
...                                                                              ...   
medical_diagnoses_Z Z95 Presence of cardiac and vascular implants a...  1.652516e-03   
                    Z96 Presence of other functional implants           7.375200e-02   
                    Z97 Presence of other devices                       4.440071e-01   
                    Z98 Other postsurgical states                       1.069124e-02   
                    Z99 Dependence on enabling machines and devices...  5.179713e-01   

                                                                        correlation  \
category            variable                                                          
Alcohol             Alcohol intake frequency.                              0.040497   
                    Average weekly red wine intake                        -0.043196   
                    Average weekly champagne plus white wine intake       -0.042765   
                    Average weekly beer plus cider intake                 -0.041572   
                    Average weekly spirits intake                         -0.038129   
...                                                                             ...   
medical_diagnoses_Z Z95 Presence of cardiac and vascular implants a...    -0.016424   
                    Z96 Presence of other functional implants             -0.009334   
                    Z97 Presence of other devices                         -0.003996   
                    Z98 Other postsurgical states                         -0.013324   
                    Z99 Dependence on enabling machines and devices...     0.003375   

                                                                        sample_size  
category            variable                                                         
Alcohol             Alcohol intake frequency.                               36693.0  
                    Average weekly red wine intake                          36686.0  
                    Average weekly champagne plus white wine intake         36688.0  
                    Average weekly beer plus cider intake                   36688.0  
                    Average weekly spirits intake                           36685.0  
...                                                                             ...  
medical_diagnoses_Z Z95 Presence of cardiac and vascular implants a...      36700.0  
                    Z96 Presence of other functional implants               36700.0  
                    Z97 Presence of other devices                           36700.0  
                    Z98 Other postsurgical states                           36700.0  
                    Z99 Dependence on enabling machines and devices...      36700.0  

[5832 rows x 3 columns]

In [ ]:
correlations = pd.DataFrame(pd.read_feather("../../data/xwas/univariate_results/linear_correlations_Abdomen.feather").to_dict.set_index("index")
correlations.index = pd.MultiIndex.from_tuples(
    list(map(eval, correlations.index.tolist())), names=["category", "variable"]
)

In [14]:
if category == "All":
    correlations = all_correlations.loc[all_correlations.index.get_level_values(0).isin(MAIN_CATEGORIES_TO_CATEGORIES[main_category])].copy()
else:
    correlations = all_correlations.loc[all_correlations.index.get_level_values(0).isin([category])].copy()

correlations.drop(index=correlations.index[(correlations["sample_size"] < 10)], inplace=True)

correlations["neg_log_p_value"] = - np.log10(correlations["p_value"])
correlations["category"] = correlations.index.get_level_values(0)
correlations["variable"] = correlations.index.get_level_values(1)

In [15]:
import plotly.express as px

fig = px.scatter(correlations, x="correlation", y="neg_log_p_value", custom_data=["variable", "p_value", "sample_size"], color="category", labels={"correlation": "Partial Correlation", "neg_log_p_value": "-log(p-value)", "category": "Categories:"}, title="Volcano plot")
fig.update_traces(
    hovertemplate="<br>".join([
        "Variable: %{customdata[0]}",
        "Partial Correlation: %{x:.3f}",
        "p-value: %{customdata[1]:.3E}",
        "Samples size: %{customdata[2]}",
    ])
)
fig.show()

In [8]:
import plotly.graph_objects as go

fig.add_trace(go.Scatter(x=[correlations["correlation"].min() - correlations["correlation"].std(), correlations["correlation"].max() + correlations["correlation"].std()], y=[-np.log10(0.05), -np.log10(0.05)], mode='lines', name = "No Correction"))

fig.add_trace(go.Scatter(x=[correlations["correlation"].min() - correlations["correlation"].std(), correlations["correlation"].max() + correlations["correlation"].std()], y=[-np.log10(0.05 / correlations.shape[0]), -np.log10(0.05 / correlations.shape[0])], mode='lines', name = "With Bonferoni Correction"))